# Example Usage of Gym-Push
 - Imports gym-push
 - Initialises the environment
 - Creates simple agent to interact with the environment

### Import and initialise

In [4]:
import gym
import gym_push
env = gym.make('push-v0')

### Training agent:
 - Set up number of episodes to loop over
 - Set up empty reward list
 - Create loop to iterate over episodes
 - Reset the environment by calling _env.reset()_, which returns the first _state_ and authorised  _action_'s (in said state)
 - Set up loop so can observe and act in the environment until the end of time (e.g. for _push-v0_ this is 3 days of notifications)
 - Take an action by calling _env.step(a)_ and passing in chosen _action_. This agent chooses an _action_ at random from the list of authorised actions returned by the environment. The values returned are the next _state_ (s1), the _reward_ (r), the _done_ flag (d) and list of authorised _action_'s for s1. 

In [5]:
'''
Random baseline agent - Initial performance: ~49.5%
- Randomly chooses an action from the list of authorised actions returned by environment
- Example final rewards for trials with 5 episodes:
        #1 [-260, -205, -170, -275, -165]
        #2 [-85, -55, -300, -185, -225]
        #3 [-165, -140, -140, -130, -200]
'''
import random
numEpisodes = 1
allRewards = []
for i in range(numEpisodes):
    print('-------------- Episode '+str(i)+' ------------------')
    episodeRewards = 0
    d = False
    s, a = env.reset()
    while not d:
        s1, r, d, a = env.step( a[random.randint(0, len(a)-1)] )
        episodeRewards += r
    allRewards.append(episodeRewards)
print(allRewards)

-------------- Episode 0 ------------------
[-140]


### Evaluating agent
The environment contains 3 lists: _shownList_, _dismissedList_ and _cachedList_. To print the contents of these lists in an easy-to-read format, simply pass the list to the _env.restoreNotification_ function which will return the list in the form of a pandas DataFrame. 
Examples of determining correct and incorrectly delivered notifications is illustrated below:

(Note: as cached notifications are possibly delivered later and in different contexts, the ground truth _action_ vale of these notifications cannot be when evaluating the agent)

In [6]:
# All notifications with action 'True' and cached False in shown list are correct
# All notifications with action 'False' and cached False in shown list are incorrect
# All notifications with cached True have no ground truth to evaluate performance..
# as context in which they are acted upon changes - can be examined to understand algorithm.

s = env.restoreNotification(env.shownList)
print('Number of shown notifications: '+str(len(s)))
print('Number of correctly shown notifications: '+str(len(s[(s.action == True) & (s.cached == False)])))
print('Number of incorrectly shown notifications: '+str(len(s[(s.action == False) & (s.cached == False)])))
print('Number of notifications cached before shown: '+str(len(s[s.cached==True])))
s[['action', 'appPackage', 'category', 'subject',
        'postedDayOfWeek', 'postedTimeOfDay', 'cached', 'numCaches',
       'action_context']].head()


Number of shown notifications: 166
Number of correctly shown notifications: 64
Number of incorrectly shown notifications: 52
Number of notifications cached before shown: 50


,action,appPackage,category,subject,postedDayOfWeek,postedTimeOfDay,cached,numCaches,action_context
0,True,com.whatsapp,msg,unknown,4,morn,False,0,
1,True,com.google.android.googlequicksearchbox,unknown,unknown,4,morn,False,0,
2,False,com.google.android.apps.maps,msg,unknown,4,morn,True,1,"[""morn"", ""4""]"
3,False,com.android.providers.downloads,msg,unknown,4,morn,False,0,
4,True,com.whatsapp,msg,unknown,4,morn,True,1,"[""morn"", ""4""]"


In [7]:
d = env.restoreNotification(env.dismissedList)
print('Number of dismissed notifications: '+str(len(d)))
print('Number of correctly dismissed notifications: '+str(len(d[(d.action == False) & (d.cached == False)])))
print('Number of incorrectly dismissed notifications: '+str(len(d[(d.action == True) & (d.cached == False)])))
print('Number of notifications cached before dismissed: '+str(len(d[d.cached==True])))
d[['action', 'appPackage', 'category', 'subject',
        'postedDayOfWeek', 'postedTimeOfDay', 'cached', 'numCaches',
       'action_context']].head()

Number of dismissed notifications: 158
Number of correctly dismissed notifications: 46
Number of incorrectly dismissed notifications: 55
Number of notifications cached before dismissed: 57


,action,appPackage,category,subject,postedDayOfWeek,postedTimeOfDay,cached,numCaches,action_context
0,False,io.wia.wia,unknown,unknown,4,morn,False,0,
1,True,com.whatsapp,unknown,unknown,4,morn,False,0,
2,True,com.whatsapp,msg,unknown,4,morn,False,0,
3,True,com.whatsapp,unknown,unknown,4,morn,True,1,"[""morn"", ""4""]"
4,True,com.whatsapp,unknown,unknown,4,morn,True,3,"[""morn"", ""4""]"


In [8]:
c = env.restoreNotification(env.cachedList)
print('Number of notifications left in cache at end: '+str(len(c)))
c[['action', 'appPackage', 'category', 'subject',
        'postedDayOfWeek', 'postedTimeOfDay', 'cached', 'numCaches',
       'action_context']].head()

Number of notifications left in cache at end: 0


,action,appPackage,category,subject,postedDayOfWeek,postedTimeOfDay,cached,numCaches,action_context
